Solving environment: ...working... 
The environment is inconsistent, please check the package plan carefully
The following packages are causing the inconsistency:

  - defaults/win-64::anaconda==2019.07=py37_0
  - defaults/win-64::numba==0.44.1=py37hf9181ef_0
failed with initial frozen solve. Retrying with flexible solve.
Solving environment: ...working... failed with repodata from current_repodata.json, will retry with next repodata source.
Solving environment: ...working... 
The environment is inconsistent, please check the package plan carefully
The following packages are causing the inconsistency:

  - defaults/win-64::anaconda==2019.07=py37_0
  - defaults/win-64::numba==0.44.1=py37hf9181ef_0
done

## Package Plan ##

  environment location: C:\ProgramData\Anaconda3

  added / updated specs:
    - autograd


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    _anaconda_depends-5.3.1    |



==> WARNING: A newer version of conda exists. <==
  current version: 4.7.12
  latest version: 4.10.3

Please update conda by running

    $ conda update -n base -c defaults conda




In [1]:
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.pyplot as plt
from matplotlib import cm
#from matplotlib.ticker import LinearLocator, FormatStrFormatter,
import matplotlib.ticker as ticker

from sklearn.linear_model import LinearRegression

import seaborn as sns
import autograd.numpy as np
from autograd import grad, elementwise_grad
import pandas as pd
from random import random, seed
from sklearn.preprocessing import PolynomialFeatures
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score, mean_squared_error
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import Lasso
from sklearn.utils import resample
from sklearn.model_selection import KFold, cross_val_score
plt.rcParams['font.size'] = 14


In [6]:
def FrankeFunction(x,y):
    term1 = 0.75*np.exp(-(0.25*(9*x-2)**2) - 0.25*((9*y-2)**2))
    term2 = 0.75*np.exp(-((9*x+1)**2)/49.0 - 0.1*(9*y+1))
    term3 = 0.5*np.exp(-(9*x-7)**2/4.0 - 0.25*((9*y-3)**2))
    term4 = -0.2*np.exp(-(9*x-4)**2 - (9*y-7)**2)

    return term1 + term2 + term3 + term4 

# Creating the design matrix, from lecture notes
def create_X(x, y):
    if len(x.shape) > 1:
        x = np.ravel(x)
        y = np.ravel(y)

    N = len(x)
    l = 2 # Number of elements in beta
    X = np.ones((N,l))
    X[:,0] = x
    X[:,1] = y
    return X


# Defining the Mean square error, from lecture notes
def CostFunction(y,ytilde):
    n = len(y)
    return 1/n * np.sum(np.abs(y-ytilde)**2)

def CostFunctionClassification(ao,target):
    n= len(ao)
    return -1/n*np.sum(target*np.log(ao) + (1-target)*np.log(1-ao))
    

def MSE(y,ytilde):
    n = len(y)
    return 1/n * np.sum(np.abs(y-ytilde)**2)

# Defining the R2 function, from lecture notes
def R2(y_data, y_model):
    return 1 - np.sum((y_data - y_model) ** 2) / np.sum((y_data - np.mean(y_data)) ** 2)

def DerivariveCostFunc(y,ytilde):
    n = len(y)
    return 2/n*(y-ytilde)

def Sigmoid(y):
    return np.exp(y)/(1+np.exp(y))

def RELU(y):
    return np.maximum(0,y)

def lexyRelu(y):
    return np.maximum(0.01*y,y)

def Identity(y):
    return y

#### From lecture notes
def Set_weights_and_bias(n_in,n_hidden,n_out):
#### Setting hiden weights
    W_hidden = np.random.randn(n_in, n_hidden)
    b_hidden = np.zeros(n_hidden) +0.01
#### setting output weights
    W_out = np.random.randn(n_hidden, n_out)
    b_out = np.zeros(n_out) +0.01
    return W_hidden, W_out, b_hidden, b_out

#### From lecture notes
def feed_forward_train(X,W_hidden, W_out, b_hidden, b_out,activation_function, output_function):
#### Hidden attac
    z_h = X@W_hidden + b_hidden
    a_h = activation_function(z_h)
#### output attac
    z_o = a_h@W_out + b_out
    a_o = output_function(z_o)
    return a_h, a_o, z_h,z_o

#### from lecture notes
def back_prop(X,Target,W_hidden, W_out, b_hidden, b_out,activation_function, output_function,hyper_par):
    a_h, a_o,z_h,z_o = feed_forward_train(X,W_hidden, W_out, b_hidden, b_out,activation_function, output_function)
#### output error
    error_out = DerivariveCostFunc(a_o,Target)*elementwise_grad(output_function,0)(z_o) #* derivative of output_function
### hidden error
    error_hiden = (error_out @ W_out.T)* elementwise_grad(activation_function,0)(z_h) #*  a_h * (1 - a_h) # the last two terms are the derivative of the sigmoid
##### the last term is the term due to the hyperparameter
    w_out_grad = a_h.T @ error_out + 2*hyper_par*W_out
    b_out_grad = np.sum(error_out, axis=0)
    
    w_hidden_grad = X.T @error_hiden+ 2*hyper_par*W_hidden
    b_hidden_grad = np.sum(error_hiden,axis=0)
    
    return w_out_grad,w_hidden_grad, b_out_grad, b_hidden_grad

###### Main Regression #######
npoints =20
x = np.sort(np.random.uniform(0, 1, npoints)) 
y = np.sort(np.random.uniform(0, 1, npoints)) 
x, y = np.meshgrid(x,y)
X = create_X(x, y)

Y = FrankeFunction(x, y) 

X_train, X_test, y_train, y_test = train_test_split(X, Y.reshape(-1,1), test_size=0.2)

numberMinibach = np.array([4,8,16,32,64])
numEpochs = np.array([10,100,1000])
#epoch_index, minibach_index = 0,3 
etas = np.logspace(-3,-1,3)
lambdas = np.logspace(-4,0,5)
numberOfStraps = 50
hyper_par = np.logspace(-6,-1,6) 
hyper = 0

epochs = 100 #numEpochs[1]
Minibach = numberMinibach[1] 
n_hiden = np.array([1,2,4,8,16])
#hiden = 8

MSE_test_ler = np.zeros((len(etas),len(hyper_par)))
MSE_train_ler = np.zeros((len(etas),len(hyper_par)))
R2_test_ler = np.zeros((len(etas),len(hyper_par)))
R2_train_ler = np.zeros((len(etas),len(hyper_par)))
counter_eta, counter_hidden =0,0
for hiden in n_hiden:
    for eta in etas:
        MSEdeglisttest = np.zeros(numberOfStraps)
        MSEdeglisttrain = np.zeros(numberOfStraps)
        R2deglisttest = np.zeros(numberOfStraps)
        R2deglisttrain = np.zeros(numberOfStraps)
        for i in range(numberOfStraps):
            w_h,w_o,b_h,b_o =Set_weights_and_bias(2,hiden,1)
            bootX,booty = resample(X_train,y_train.reshape(-1,1))
            MiniBachSize = int(bootX.shape[0]/Minibach)
            for e in range(epochs):
                for j in range(Minibach):
                    miniBach = np.random.randint(Minibach)
                    miniBachMin, miniBachMax = MiniBachSize * miniBach,(MiniBachSize) * (miniBach+1)
                    #a_h,a_o = feed_forward_train(bootX[miniBachMin: miniBachMax],w_h,w_o,b_h,b_o,Sigmoid,Identity)
                    w_out_grad,w_hidden_grad,b_out_grad, b_hidden_grad = back_prop(
                        bootX[miniBachMin: miniBachMax],booty[miniBachMin: miniBachMax],w_h, w_o, b_h, b_o,Sigmoid, Identity,hyper)
                    w_h -= eta*w_hidden_grad
                    w_o -= eta*w_out_grad
                    b_h -= eta*b_hidden_grad
                    b_o -= eta*b_out_grad
            a_h, y_pred_test,z_h,z_o =feed_forward_train(X_test,w_h,w_o,b_h,b_o,Sigmoid,Identity)
            a_h, y_pred_train,z_h,z_o =feed_forward_train(X_train,w_h,w_o,b_h,b_o,Sigmoid,Identity)
            MSEdeglisttest[i] =MSE(y_test,y_pred_test)
            MSEdeglisttrain[i]= MSE(y_train,y_pred_train)
            R2deglisttest[i] = R2(y_test,y_pred_test)
            R2deglisttrain[i] = R2(y_train,y_pred_train)
        MSE_train_ler[counter_eta,counter_hidden] = np.mean(MSEdeglisttrain)
        MSE_test_ler[counter_eta,counter_hidden] = np.mean(MSEdeglisttest)
        R2_test_ler[counter_eta,counter_hidden] = np.mean(R2deglisttest)
        R2_train_ler[counter_eta,counter_hidden] = np.mean(R2deglisttrain)
        counter_eta +=1
        print(counter_eta)
    counter_hidden +=1
    counter_eta =0

tick = ticker.ScalarFormatter(useOffset=False, useMathText=True)
tick.set_powerlimits((0,0))

tx = [u"${}$".format(tick.format_data(x)) for x in n_hiden]
ty = [u"${}$".format(tick.format_data(x)) for x in etas]

#lambdas_sea = [lambdas[i] for i in range(len(lambdas))]
#etas_sea =  [etas[i] for i in range(len(etas))]

fig, ax = plt.subplots(figsize = (10, 10))
sns.heatmap(data=MSE_test_ler,ax=ax, cmap="viridis",annot=True ,xticklabels=tx, yticklabels=ty,)
ax.set_xlabel(r'$\lambda$')
ax.set_ylabel(r'$\eta$')
plt.tight_layout()
#plt.savefig(f"Test_MSE_Single_hidden_function_of_hyper_par_RELU.pdf")
plt.show()

fig, ax = plt.subplots(figsize = (10, 10))
sns.heatmap(data=R2_test_ler,ax=ax, cmap="viridis",annot=True ,xticklabels=tx, yticklabels=ty,)
ax.set_xlabel(r'$\lambda$')
ax.set_ylabel(r'$\eta$')
plt.tight_layout()
#plt.savefig(f"Test_R2_Single_hidden_function_of_hyper_par_RELU.pdf")
plt.show()


fig, ax = plt.subplots(figsize = (10, 10))
sns.heatmap(data=MSE_train_ler,ax=ax, cmap="viridis",annot=True ,xticklabels=tx, yticklabels=ty,)
ax.set_xlabel(r'$\lambda$')
ax.set_ylabel(r'$\eta$')
plt.tight_layout()
#plt.savefig(f"Train_MSE_Single_hidden_function_of_hyper_par_RELU.pdf")
plt.show()

fig, ax = plt.subplots(figsize = (10, 10))
sns.heatmap(data=R2_train_ler,ax=ax, cmap="viridis",annot=True ,xticklabels=tx, yticklabels=ty,)
ax.set_xlabel(r'$\lambda$')
ax.set_ylabel(r'$\eta$')
plt.tight_layout()
#plt.savefig(f"Train_R2_Single_hidden_function_of_hyper_par_RELU.pdf")
plt.show()

KeyboardInterrupt: 